## CM3 DATA 가공하기 

In [2]:

# CM3 data로 복습하기 1 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats 
import sweetviz as sv
from statsmodels.formula.api import ols
import datetime as dt 

import warnings
warnings.filterwarnings("ignore")

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 분석 연습 파일 불러오기
cm3 = pd.read_excel("d:/novelis/ds2023/CM3_분석연습.xls")
cm3.head(3)

,구분,회수율,시작일시,종료일시,준비시간,작업시간,소요시간(UPTIME),Batch No,YS_TARGET_온도,Make as Alloy,...,GAUGE 3SIGMA,FLATNESS 3SIGMA,ENTRY WEIGHT,2STD 평균속도,2STD 최대속도,2STD BENDING,2STD ROLL FORCE,입측 COIL온도,평균 출측 온도,COIL TEMP DIFF
0,NaN,97.21,2023-01-01 06:27:25,2023-01-01 06:37:00,2.83,6.75,9.58,S843331,165.0,M3104,...,0.82,2.76,11881.0,1109.0,1392.0,14.3,489.0,58.0,142.53360,-22.46640
1,NaN,97.30,2023-01-01 06:37:00,2023-01-01 06:46:01,2.73,6.28,9.01,S843341,165.0,M3104,...,0.78,3.12,11876.0,1180.0,1518.0,16.7,485.0,58.0,142.68825,-22.31175
2,NaN,97.47,2023-01-01 06:46:00,2023-01-01 06:56:10,2.87,7.28,10.15,S841842,170.0,K3104,...,0.77,3.37,12312.0,1242.0,1569.0,38.9,442.0,62.0,142.10706,-27.89294


In [4]:
cm3_a = cm3[['구분','Batch No', '회수율', '시작일시', '종료일시', '준비시간', '작업시간', '소요시간(UPTIME)']]
cm3_b = cm3[['Batch No','시작일시', '종료일시','YS_TARGET_온도', 'Make as Alloy', 'Pass','폭', 'ENTRY GAUGE', 'INTER GAUGE', 'EXIT GAUGE']]
cm3_c = cm3[['Batch No','시작일시', '종료일시','2STD BENDING','2STD 평균속도', '2STD 최대속도', '2STD ROLL FORCE', '입측 COIL온도', '평균 출측 온도']]

In [5]:
# 날짜 바꾸기

# A조: 06:30 ~ 14:30 / B조: 14:30 ~ 22:30 / C조: 22:30 ~ 06:30
# 예: 2023-01-01 05:30:00 작업 이력은 2022-12-31의 작업분 
# 따라서 Day 표시에는 1일이 아닌 31일이어야함 


In [6]:
cm3_b.set_index("시작일시", inplace = True) 

In [7]:
cm3_b.head(3)

,Batch No,종료일시,YS_TARGET_온도,Make as Alloy,Pass,폭,ENTRY GAUGE,INTER GAUGE,EXIT GAUGE
시작일시,,,,,,,,,
2023-01-01 06:27:25,S843331,2023-01-01 06:37:00,165.0,M3104,F,1755.0,1.907,0.729,0.271
2023-01-01 06:37:00,S843341,2023-01-01 06:46:01,165.0,M3104,F,1755.0,1.905,0.729,0.271
2023-01-01 06:46:00,S841842,2023-01-01 06:56:10,170.0,K3104,F,1575.0,2.012,0.735,0.261


In [8]:
# Timedelta: 시간 또는 날짜의 차이를 계산하는 method
# 단, 한번만 동작 후 비활성화로 바꿔야함. 

# cm3_b.index = cm3_b.index - pd.Timedelta(hours=6, minutes=30) 

In [9]:
cm3_b.head(3)

,Batch No,종료일시,YS_TARGET_온도,Make as Alloy,Pass,폭,ENTRY GAUGE,INTER GAUGE,EXIT GAUGE
시작일시,,,,,,,,,
2022-12-31 23:57:25,S843331,2023-01-01 06:37:00,165.0,M3104,F,1755.0,1.907,0.729,0.271
2023-01-01 00:07:00,S843341,2023-01-01 06:46:01,165.0,M3104,F,1755.0,1.905,0.729,0.271
2023-01-01 00:16:00,S841842,2023-01-01 06:56:10,170.0,K3104,F,1575.0,2.012,0.735,0.261


In [10]:
cm3_b["Year"]  = cm3_b.index.year
cm3_b["Month"] = cm3_b.index.month
cm3_b["Day"]   = cm3_b.index.day 

In [12]:
# 데이터 엑셀로 저장하기

# cm3_b.to_excel(excel_writer ="d:/ds2023/data_processing_2023_0831_1600.xlsx",
#                sheet_name   ="cm3_b",
#                header       = True,
#                index        = False)

In [19]:
c1 = cm3_b.resample('D')['폭'].mean().round(0)
c1

시작일시
2022-12-31    1755.0
2023-01-01    1717.0
2023-01-02    1745.0
2023-01-03    1681.0
2023-01-04    1696.0
               ...  
2023-08-21    1755.0
2023-08-22    1743.0
2023-08-23    1775.0
2023-08-24    1747.0
2023-08-25    1672.0
Freq: D, Name: 폭, Length: 238, dtype: float64

In [24]:
# 전체 기간에서의 폭 트렌드 데이터 확인

# sns.barplot(x = cm3_b.index,
#             y = cm3_b["폭"])